# Spark Configuration

In [1]:
import numpy
import pandas
import pyproj
import shapely
import sklearn

In [2]:
print(numpy.__version__)
print(pandas.__version__)
print(pyproj.__version__)
print(shapely.__version__)
print(sklearn.__version__)

1.26.4
1.5.3
3.7.0
2.0.6
1.5.2


In [3]:
# Import libraries
import h3
import h3_pyspark
import pandas as pd
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lag, lead
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, TimestampType, StringType, DateType
from datetime import datetime, timedelta



pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

In [4]:
import math
import os
from functools import reduce
from math import asin, atan2, cos, degrees, floor, radians, sin, sqrt

import geopandas as gpd
import geopy
import h3
import numpy as np
import pandas as pd
import pyspark
import sedona
# import rasterio
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window

from shapely.wkt import loads

print("pandas : ", pd.__version__)
print("geopandas : ", gpd.__version__)
print("pyspark : ", pyspark.__version__)
print("sedona : ", sedona.version)

pandas :  1.5.3
geopandas :  1.0.1
pyspark :  3.3.4
sedona :  1.2.1


In [5]:
os.environ["PYSPARK_PYTHON"] = "./env/bin/python"
os.environ["HADOOP_CONF_DIR"] = "/etc/spark3/conf.cloudera.spark3_on_yarn/yarn-conf"
os.environ["HADOOP_HOME"] = "/opt/cloudera/parcels/CDH/lib/hadoop"
os.environ["SPARK_HOME"] = "/opt/cloudera/parcels/SPARK3-3.3.2.3.3.7190.4-1-1.p0.51021169/lib/spark3"
os.environ["SPARK_CONF_DIR"] = "/etc/spark3/conf.cloudera.spark3_on_yarn"

conf = (
    SparkConf()
    .setMaster("yarn")
    .setAppName("data-cerdas-november")
    .set("spark.dynamicAllocation.maxExecutors", "100")
    .set("spark.dynamicAllocation.minExecutors", "1")
    .set("spark.executor.cores", "12")
    .set("spark.executor.memory", "64g")
    .set("spark.sql.shuffle.partitions", "7000")
    .set("spark.yarn.queue", "root.s_project.hue_project")
    .set("spark.driver.maxResultSize","8g")
    .set(
        "spark.yarn.appMasterEnv.PYSPARK_PYTHON",
        "./env/bin/python",
    )
    .set(
        "spark.yarn.dist.archives", "hdfs://nsdiscovery/warehouse/tablespace/external/hive/pnt_bps_int.db/envs/mobility_310.tar.gz#env"
    )
)

In [6]:
sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/25 04:23:49 WARN  util.Utils: [Thread-3]: Service 'sparkDriver' could not bind on port 30060. Attempting port 30061.
25/03/25 04:23:50 WARN  util.Utils: [Thread-3]: Service 'SparkUI' could not bind on port 30072. Attempting port 30073.
25/03/25 04:23:52 WARN  conf.HiveConf: [Thread-3]: HiveConf of name hive.metastore.runworker.in does not exist
25/03/25 04:23:52 WARN  conf.HiveConf: [Thread-3]: HiveConf of name hive.masking.algo does not exist
25/03/25 04:23:56 WARN  util.Utils: [Thread-3]: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.
25/03/25 04:23:57 WARN  ipc.Client: [Thread-3]: Exception encountered while connecting to the server : org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.ipc.StandbyException): Operation category READ is not supported in state st

# Function

In [7]:
def haversine_distance(lon1, lat1, lon2, lat2):
    R = 6371.0  # Earth's radius in kilometers
    dlat = f.radians(lat2 - lat1)
    dlon = f.radians(lon2 - lon1)
    a = f.pow(f.sin(dlat/2), 2) + \
        f.cos(f.radians(lat1)) * f.cos(f.radians(lat2)) * f.pow(f.sin(dlon/2), 2)
    c = 2 * f.atan2(f.sqrt(a), f.sqrt(1-a))
    return f.round(R * c, 2)

def to_parent_res9(h3_index):
    if h3_index != None:
        h3_index = str(h3_index)
        return h3.cell_to_parent(h3_index, 9)

def to_parent_res8(h3_index):
    if h3_index != None:
        h3_index = str(h3_index)
        return h3.cell_to_parent(h3_index, 8)

def to_parent_res7(h3_index):
    if h3_index != None:
        h3_index = str(h3_index)
        return h3.cell_to_parent(h3_index, 7)

geo_to_h3 = F.udf(h3.latlng_to_cell)

In [8]:
def data_cerdas_trip_mobility_trip(
    staypoint,
    inbound,
    usual_env,
    jak_zone,
    jak_poi,
    lau_ref,
    event,
    lau,
    cfd,
    start_date,
    end_date,
    suffix
):
    
    operator_name = 'TSEL'
    
    staypoint = staypoint.filter(F.col('event_date')<=end_date)
    staypoint = staypoint.filter(F.col('event_date')>=start_date)
    
    staypoint = staypoint.filter(F.substring('hash_msisdn',-1,1)==suffix)
    staypoint = staypoint.filter((F.col('latitude').isNotNull())&(F.col('longitude').isNotNull()))
    
    inbound = inbound.filter(F.col('event_date')<=end_date)
    inbound = inbound.filter(F.col('event_date')>=start_date)
    
    inbound = inbound.filter(F.substring('imsi',-1,1)==suffix)
    inbound = inbound.filter((F.col('latitude').isNotNull())&(F.col('longitude').isNotNull()))
    
    staypoint = staypoint\
                .withColumnRenamed('hash_msisdn', 'msisdn')\
                .withColumnRenamed('start_time', 'start_datetime')\
                .withColumnRenamed('end_time', 'end_datetime')\
                .withColumn('latitude', f.round(col('latitude').cast('double'),5))\
                .withColumn('longitude', f.round(col('longitude').cast('double'),5))\
                .dropDuplicates(['msisdn','start_datetime','end_datetime','longitude','latitude'])\
                .withColumn('operator', f.lit(operator_name))\
                .withColumn('country', f.lit('INDONESIA'))\
                .withColumn('event_month', f.to_date(col('end_datetime')))\
                .withColumn('event_month', f.format_string("%04d-M%02d", f.year(col('event_month')), f.month(col('event_month'))) )\
                .withColumn('h3_10', geo_to_h3('latitude', 'longitude', f.lit(10)))\
                .withColumn('h3_9', to_parent_res9_udf(col('h3_10')))\
                .withColumn('h3_8', to_parent_res8_udf(col('h3_9')))\
                .withColumn('h3_7', to_parent_res7_udf(col('h3_8')))\
                .drop('event_date') \
                .select(['msisdn','latitude','longitude','start_datetime','end_datetime','operator','country','event_month','h3_7','h3_9','h3_10'])

    part0_1 = Window.partitionBy(['msisdn']).orderBy(['datetime'])
    part0_2 = Window.partitionBy(['msisdn']).orderBy(['datetime']).rowsBetween(Window.unboundedPreceding, Window.currentRow)
    part0_3 = Window.partitionBy(['msisdn','stop_movement']).orderBy(['datetime'])
    part0_4 = Window.partitionBy(['msisdn','stop_movement']).orderBy(['datetime']).rowsBetween(Window.unboundedPreceding, Window.currentRow)
    part0_5 = Window.partitionBy(['msisdn','stop_movement','spot_movement'])

    inbound = inbound\
                .withColumnRenamed('imsi', 'msisdn')\
                .withColumnRenamed('hash_msisdn', 'msisdn')\
                .withColumnRenamed('msisdn_encrypted', 'msisdn')\
                .withColumn('latitude', f.round(col('latitude').cast('double'),5))\
                .withColumn('longitude', f.round(col('longitude').cast('double'),5))\
                .dropDuplicates(['msisdn','datetime','longitude','latitude'])\
                .withColumn('lag_latitude', f.lag('latitude').over(part0_1))\
                .withColumn('lag_longitude', f.lag('longitude').over(part0_1))\
                .withColumn('lag_datetime', f.lag('datetime').over(part0_1))\
                .withColumn('time_interval', f.unix_timestamp('datetime') - f.unix_timestamp('lag_datetime')) \
                .withColumn('time_interval_hour', f.round(col('time_interval')/3600, 2) )\
                .withColumn('distance',  haversine_distance(col('lag_longitude'), col('lag_latitude'), col('longitude'), col('latitude')) )\
                .withColumn('speed', f.round( col('distance') / (col('time_interval') / 3600) ,2) )\
                .withColumn('speed_raise', f.when( col('speed') >= 10.0, 1 ).otherwise(0) )\
                .withColumn('lag_speed_raise', f.lag('speed_raise').over(part0_1) )\
                .withColumn('speed_gap', f.when( col('lag_speed_raise') != col('speed_raise'), 1 ).otherwise(0) )\
                .withColumn('max_duration', f.when( col('time_interval_hour') >= 8.0, 1 ).otherwise(0) )\
                .withColumn('stop_gap', f.when( ( col('speed_gap') == 1 ) | ( col('max_duration') == 1 ), 1 ).otherwise(0) )\
                .withColumn('stop_movement', f.sum('stop_gap').over(part0_2))\
                .withColumn('h3_10', geo_to_h3('latitude', 'longitude', f.lit(10)))\
                .withColumn('h3_9', to_parent_res9_udf(col('h3_10')))\
                .withColumn('h3_8', to_parent_res8_udf(col('h3_9')))\
                .withColumn('h3_7', to_parent_res7_udf(col('h3_8')))\
                .withColumn('lag_h3_10', f.lag('h3_10').over(part0_3) )\
                .withColumn('spot_gap', f.when( (col('lag_h3_10') != col('h3_10')), 1 ).otherwise(0) )\
                .withColumn('spot_movement', f.sum('spot_gap').over(part0_4))\
                .withColumn('x', f.cos(f.radians('latitude')) * f.cos(f.radians('longitude')) )\
                .withColumn('y', f.cos(f.radians('latitude')) * f.sin(f.radians('longitude')) )\
                .withColumn('z', f.sin(f.radians('latitude')) )\
                .withColumn('total', f.count('spot_movement').over(part0_5) )\
                .withColumn('x_total', f.sum('x').over(part0_5) / col('total') )\
                .withColumn('y_total', f.sum('y').over(part0_5) / col('total') )\
                .withColumn('z_total', f.sum('z').over(part0_5) / col('total') )\
                .withColumn('sqrt', f.sqrt(col('x_total') * col('x_total') + col('y_total') * col('y_total') ) )\
                .withColumn('centroid_lat', f.round(f.degrees(f.atan2(F.col('z_total'), F.col('sqrt'))),6) )\
                .withColumn('centroid_lon', f.round(f.degrees(f.atan2(F.col('y_total'), F.col('x_total'))),6) )\
                .withColumn('start_datetime', f.min('datetime').over(part0_5) )\
                .withColumn('end_datetime', f.max('datetime').over(part0_5) )\
                .withColumn('event_month', f.to_date(col('end_datetime')))\
                .withColumn('event_month', f.format_string("%04d-M%02d", f.year(col('event_month')), f.month(col('event_month'))) )\
                .withColumn('operator',F.lit(operator_name)) \
                .select(['msisdn','latitude','longitude','start_datetime','end_datetime','operator','country','event_month','h3_7','h3_9','h3_10'])

    inbound = inbound.dropDuplicates(inbound.columns)
    
    master_raw = staypoint.union(inbound)
    
    jak_poi = jak_poi\
          .withColumnRenamed("id", "poi_id")\
          .select(['poi_id','poi','h3_10'])
    
    event = event.withColumn('h3_7_event', to_parent_res7_udf(col('h3_10'))) # Perubahan h3
    
    # Tambahan CFD
    cfd = cfd.withColumn('h3_7_cfd',to_parent_res7_udf(col('h3_10')))

    lau = lau.withColumn('h3_7', to_parent_res7_udf(col('h3_10')))\
      .drop('h3_10')\
      .dropDuplicates(['h3_7'])

    lau_ref =  lau_ref\
                .withColumn("kdkab", f.lpad(col("kdkab"), 2, "0"))\
                .withColumn("kdkec", f.lpad(col("kdkec"), 3, "0"))\
                .withColumn('idkab', f.concat_ws('|', 'kdprov', 'kdkab'))\
                .withColumn('idkec', f.concat_ws('|', 'kdprov', 'kdkab', 'kdkec'))

    # Join datasets
    # master_raw = master_raw.join(lau, ['h3_10'], 'left')
    master_raw = master_raw.join(lau, ['h3_7'], 'left') # join h3_7

    master_raw = master_raw\
                    .withColumn('activity_kab', f.col('kab')) \
                    .withColumn('activity_kec', f.col('kec')) \
                    .withColumn('activity_prov', f.substring('kab',1,2)) \
                    .withColumn('activity_prov', (f.col('activity_prov').cast('int')).cast('string'))\
                    .drop("kab","kec")

    # Join datasets
    master_raw = master_raw.join(jak_poi, ['h3_10'], 'left')

    # Marking Jak POI area
    master_raw = master_raw\
                  .withColumn('h3_7_poi', f.when( col('poi_id').isNotNull(), col('poi_id'))\
                              .otherwise(col('h3_7')))
    
    # MOBILITY 
    usual_env = usual_env\
            .join(lau_ref.select(['idkec', 'nmkab', 'nmkec']),
                    usual_env.home_kec==lau_ref.idkec, how='left'
                  )\
            .drop('idkec')\
            .withColumn('home_h3_7', to_parent_res7_udf(col('home_h3_8')))\
            .withColumn('work_h3_7', to_parent_res7_udf(col('work_h3_8')))\
            .drop('home_h3_8', 'work_h3_8')\
            .withColumnRenamed('nmkab','home_nmkab')\
            .withColumnRenamed('nmkec','home_nmkec')
    
#     usual_env = usual_env\
#                 .join(lau_ref.select(['idkec', 'nmkab', 'nmkec']),
#                         usual_env.home_kec==lau_ref.idkec, how='left'
#                       )\
#                 .drop('idkec')\
#                 .withColumnRenamed('nmkab','home_nmkab')\
#                 .withColumnRenamed('nmkec','home_nmkec')\
#                 .withColumn('event_month', f.to_date(f.col('event_month'))) \
#                 .withColumn('event_month', f.format_string("%04d-M%02d", f.year(col('event_month')), f.month(col('event_month'))) ) ## reformatting event_month

    mobility = master_raw\
            .join(usual_env, ['msisdn', 'event_month'], 'left')\
            .select(['event_month','msisdn','home_kab','home_nmkab','home_kec','home_nmkec','work_kab','home_h3_7','work_h3_7',
                     'operator','country','latitude','longitude',
                     'nmprov','nmkab','nmkec','activity_prov','activity_kab','activity_kec','h3_10','h3_9','h3_7',
                     'h3_7_poi','poi_id','poi','start_datetime','end_datetime'])\
            .withColumn('home_kab', f.when( (col('home_kab').isNull()) & (col('country')!='INDONESIA'), col('country')).otherwise(col('home_kab')) )\
            .withColumn('work_kab', f.when( (col('work_kab').isNull()) & (col('country')!='INDONESIA'), col('country')).otherwise(col('work_kab')) )\
            .withColumn('home_h3_7', f.when( (col('home_h3_7').isNull()) & (col('country')!='INDONESIA'), col('country')).otherwise(col('home_h3_7')) )\
            .withColumn('work_h3_7', f.when( (col('work_h3_7').isNull()) & (col('country')!='INDONESIA'), col('country')).otherwise(col('work_h3_7')) )\
            .withColumn('country', f.upper(col('country')))
    
    # MOBILITY 0
    part1 = Window.partitionBy(['msisdn']).orderBy(['start_datetime'])

    # Filter record that have usual environment
    max_speed = 1061
    mobility_0 = mobility\
                    .filter( col('home_h3_7').isNotNull() & col('work_h3_7').isNotNull() )\
                    .filter( col('h3_7').isNotNull() )

    # Remove speed more than 1061kmh (fast removal)
    mobility_0 = mobility_0\
                  .withColumn('prev_latitude', f.lag('latitude').over(part1))\
                  .withColumn('prev_longitude', f.lag('longitude').over(part1))\
                  .withColumn('prev_end_datetime', f.lag('end_datetime').over(part1))\
                  .withColumn('distance',  haversine_distance(col('prev_longitude'), col('prev_latitude'), col('longitude'), col('latitude')) )\
                  .withColumn('time_seconds', f.unix_timestamp('start_datetime') - f.unix_timestamp('prev_end_datetime')) \
                  .withColumn('speed', f.round( col('distance') / (col('time_seconds') / 3600), 2) )\
                  .filter(col('speed') <= max_speed) #max airplane speed
    
    # MOBILITY 1
    
    part2 = Window.partitionBy(['msisdn']).orderBy(['start_datetime']).rowsBetween(Window.unboundedPreceding, Window.currentRow)
    part3 = Window.partitionBy(['msisdn','move_trip'])

    trip_gap_inbound = 24.0
    trip_hour_mark = 6.0

    # Define trip
    mobility_1_trip = mobility_0\
                      .withColumn('category', f.when(col('country')!='INDONESIA', 'Inbound').otherwise('Domestic'))\
                      .withColumn('interval_hours', f.round(col('time_seconds')/3600,1))\
                      .withColumn('trip_gap', f.when((col('category')=='Inbound') & (col('interval_hours') > trip_gap_inbound), 1).otherwise(0))\
                      .withColumn('flag_env', f.when( (col('h3_7') == col('home_h3_7')) | (col('h3_7') == col('work_h3_7')), 1 ).otherwise(0))\
                      .withColumn('prev_env', f.lag('flag_env').over(part1))\
                      .withColumn('next_env', f.lead('flag_env').over(part1))\
                      .withColumn('flag_move_env', f.when(col('flag_env') != col('prev_env'), 1).otherwise(0) )\
                      .withColumn('move_trip', f.when(col('category')=='Inbound', f.sum('trip_gap').over(part2))\
                                              .otherwise(f.sum('flag_move_env').over(part2)) )\
                      .withColumn('start_trip_datetime', f.first('start_datetime').over(part3))\
                      .withColumn('end_trip_datetime', f.last('end_datetime').over(part3))\
                      .withColumn('interval_trip_seconds', f.to_timestamp(col('end_trip_datetime')).cast('double') - f.to_timestamp(col('start_trip_datetime')).cast('double') )\
                      .withColumn('interval_trip_hour', f.round(col('interval_trip_seconds')/3600, 2) )\
                      .withColumn('interval_trip_day', f.datediff(col('end_trip_datetime'), col('start_trip_datetime')) )
    
    # Take trip only which at least 2 hours duration
    mobility_1_trip = mobility_1_trip\
                        .filter( col('interval_trip_hour') >= trip_hour_mark )
    
    mobility_1_trip = mobility_1_trip.withColumn('suffix',F.substring('msisdn',-1,1))
    mobility_1_trip = mobility_1_trip.withColumn('execute_month',F.substring(F.lit(end_date),1,7))
    return mobility_1_trip

In [9]:

to_parent_res9_udf = f.udf(to_parent_res9, StringType())
to_parent_res8_udf = f.udf(to_parent_res8, StringType())
to_parent_res7_udf = f.udf(to_parent_res7, StringType())

operator_name = 'TSEL'

# Atur Bulan di Sini

In [10]:
# !hdfs dfs -rm -r hdfs://nsdiscovery/warehouse/tablespace/external/hive/pnt_bps_int.db/data_cerdas_mobility_trip/execute_month=2024-11

In [11]:
# staypoint = spark.read.table('p_gsa.mobility_staypoint_dd')
staypoint = spark.read.table('pe_bps.wisnus_sample5_staypoint_2024')
# inbound = spark.read.table('pnt_bps_int.kemenpar_inbound_cross_border_dd')
inbound = spark.read.table('pnt_bps_int_stg.inbound_sample5_lbs_2024')

usual_env = spark.read.table('pnt_bps_int.data_cerdas_ue_sample')

jak_zone = pd.read_csv('data/jak_zone.csv')
jak_zone = spark.createDataFrame(jak_zone)

jak_poi = pd.read_csv('data/jak_poi.csv')
jak_poi = spark.createDataFrame(jak_poi)

lau_ref = pd.read_csv('data/lau_ref.csv')
lau_ref = spark.createDataFrame(lau_ref)

event = pd.read_csv('data/event.csv')
event = spark.createDataFrame(event)

cfd = pd.read_csv('data/cfd.csv')
cfd = spark.createDataFrame(cfd)

lau = spark.read.table('pe_bps.indonesia_h3_38prov')
lau = lau.select(['h3_10','kab','kec','nmprov','nmkab','nmkec'])

start_date = '2024-06-01' # 6 bulan kebelakang
end_date = '2024-11-30' # bulan yang di running

suffix = '8'

25/03/25 04:24:24 WARN  conf.HiveConf: [Thread-3]: HiveConf of name hive.metastore.runworker.in does not exist
25/03/25 04:24:24 WARN  conf.HiveConf: [Thread-3]: HiveConf of name hive.masking.algo does not exist
25/03/25 04:24:24 WARN  client.HiveClientImpl: [Thread-3]: Detected HiveConf hive.execution.engine is 'tez' and will be reset to 'mr' to disable useless hive logic
Hive Session ID = 1371c378-e782-43c9-a2d3-fbfc3215a9fd
25/03/25 04:24:26 WARN  conf.HiveConf: [Thread-3]: HiveConf of name hive.metastore.runworker.in does not exist
25/03/25 04:24:26 WARN  conf.HiveConf: [Thread-3]: HiveConf of name hive.masking.algo does not exist
/home/tiana-collab/.conda/envs/mobility310/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/tiana-collab/.conda/envs/mobility310/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:4

In [12]:
mobility_trip = data_cerdas_trip_mobility_trip(
    staypoint,
    inbound,
    usual_env,
    jak_zone,
    jak_poi,
    lau_ref,
    event,
    lau,
    cfd,
    start_date,
    end_date,
    suffix
)

In [13]:
# %%time
# mobility_trip.repartition(100).write.partitionBy('execute_month','event_month','suffix').format('parquet').mode('append').saveAsTable('pnt_bps_int.data_cerdas_mobility_trip')

# Looping per Suffix

In [14]:
import logging

# Set up logging
logging.basicConfig(filename='logs/logging_mobility_trip_sample.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Example usage
logging.info('This is an info message.')
logging.info('Another info message.')

In [15]:
for i in range(0,10):
    suffix = str(i) 
    logging.info(f'PROCESS SUFFIX: {suffix}')
    mobility_trip = data_cerdas_trip_mobility_trip(
        staypoint,
        inbound,
        usual_env,
        jak_zone,
        jak_poi,
        lau_ref,
        event,
        lau,
        cfd,
        start_date,
        end_date,
        suffix
    )
    mobility_trip.repartition(100).write.mode('append').partitionBy('execute_month','event_month','suffix').saveAsTable('pnt_bps_int_stg.data_cerdas_mobility_trip_sample')

25/03/25 04:24:49 WARN  util.package: [Thread-3]: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/03/25 04:28:45 WARN  conf.HiveConf: [SparkCatalogEventProcessor-thread]: HiveConf of name hive.metastore.runworker.in does not exist
25/03/25 04:28:45 WARN  conf.HiveConf: [SparkCatalogEventProcessor-thread]: HiveConf of name hive.masking.algo does not exist
25/03/25 04:28:47 WARN  sql.CommandsHarvester$: [SparkExecutionPlanProcessor-thread]: Missing unknown leaf node: LogicalRDD [id#82, poi#83, category#84, nmprov#85, kdprov#86L, nmkab#87, kdkab#88L, h3_10#89], false

25/03/25 04:28:47 WARN  sql.CommandsHarvester$: [SparkExecutionPlanProcessor-thread]: Missing unknown leaf node: LogicalRDD [nmprov#98, nmkab#99, nmkec#100, kdprov#101L, kdkab#102L, kdkec#103L], false

25/03/25 04:28:48 WARN  producer.ProducerConfig: [Atlas Notifier 0]: These configurations '[key.deserializer, value.deseria

# Checking

In [16]:
q = """SHOW PARTITIONS pnt_bps_int_stg.data_cerdas_mobility_trip_sample"""
spark.sql(q).toPandas()[-20:]

25/03/25 05:08:10 WARN  sql.CommandsHarvester$: [SparkExecutionPlanProcessor-thread]: Missing unknown leaf node: LogicalRDD [nmprov#98, nmkab#99, nmkec#100, kdprov#101L, kdkab#102L, kdkec#103L], false



partition
49  execute_month=2024-11/event_month=2024-M10/suf...
50  execute_month=2024-11/event_month=2024-M10/suf...
51  execute_month=2024-11/event_month=2024-M10/suf...
52  execute_month=2024-11/event_month=2024-M10/suf...
53  execute_month=2024-11/event_month=2024-M10/suf...
54  execute_month=2024-11/event_month=2024-M10/suf...
55  execute_month=2024-11/event_month=2024-M10/suf...
56  execute_month=2024-11/event_month=2024-M10/suf...
57  execute_month=2024-11/event_month=2024-M10/suf...
58  execute_month=2024-11/event_month=2024-M10/suf...
59  execute_month=2024-11/event_month=2024-M11/suf...
60  execute_month=2024-11/event_month=2024-M11/suf...
61  execute_month=2024-11/event_month=2024-M11/suf...
62  execute_month=2024-11/event_month=2024-M11/suf...
63  execute_month=2024-11/event_month=2024-M11/suf...
64  execute_month=2024-11/event_month=2024-M11/suf...
65  execute_month=2024-11/event_month=2024-M11/suf...
66  execute_month=2024-11/event_month=2024-M11/suf...
67  execute_month=2024-11/event_month=2024-M11/suf...
68  execute_month=2024-11/event_month=2024-M11/suf...

In [17]:
df = spark.read.table('pnt_bps_int_stg.data_cerdas_mobility_trip_sample')
df = df.filter(F.col('home_nmkab').isNotNull())
df.limit(10).toPandas()

25/03/25 05:08:11 WARN  server.TransportChannelHandler: [rpc-server-6-2]: Exception in connection from /10.53.177.95:49330
java.io.IOException: Connection reset by peer
	at sun.nio.ch.FileDispatcherImpl.read0(Native Method) ~[?:?]
	at sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39) ~[?:?]
	at sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:276) ~[?:?]
	at sun.nio.ch.IOUtil.read(IOUtil.java:233) ~[?:?]
	at sun.nio.ch.IOUtil.read(IOUtil.java:223) ~[?:?]
	at sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:356) ~[?:?]
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:254) ~[netty-buffer-4.1.100.Final.jar:4.1.100.Final]
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1132) ~[netty-buffer-4.1.100.Final.jar:4.1.100.Final]
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:357) ~[netty-transport-4.1.100.Final.jar:4.1.100.Final]
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByt

msisdn home_kab               home_nmkab   home_kec  \
0  -1204272110084355666    53|01              SUMBA BARAT  53|01|050   
1   2634358465267132396    21|01                  KARIMUN  21|01|031   
2  -5247457192398318316    33|09                 BOYOLALI  33|09|040   
3   2634358465267132396    21|01                  KARIMUN  21|01|031   
4  -3053795097724068166    16|09  OGAN KOMERING ULU TIMUR  16|09|080   
5   6774139058896240176    32|75                   BEKASI  32|75|061   
6   8002608135689740776    35|15                 SIDOARJO  35|15|120   
7  -5790789699563050956    12|78             GUNUNGSITOLI  12|78|010   
8   7943191096088838036    36|71                TANGERANG  36|71|020   
9   2764118417862802676    35|02                 PONOROGO  35|02|130   

            home_nmkec work_kab        home_h3_7        work_h3_7 operator  \
0                 LOLI    53|01  8795ad751ffffff  8795ad751ffffff     TSEL   
1                 BURU    21|01  876520830ffffff  876520830ffffff     TSEL   
2                MUSUK    33|09  878d8c024ffffff  878d8c024ffffff     TSEL   
3                 BURU    21|01  876520830ffffff  876520830ffffff     TSEL   
4          BELITANG II    16|09  878ca941dffffff  878ca941dffffff     TSEL   
5         MEDAN SATRIA    32|16  878c104ceffffff  878c104c3ffffff     TSEL   
6              BUDURAN    35|15  878d842d8ffffff  878d842d8ffffff     TSEL   
7  GUNUNGSITOLI IDANOI    12|78  876574758ffffff  876574758ffffff     TSEL   
8             CIPONDOH    36|71  878c106f0ffffff  878c106f0ffffff     TSEL   
9               JAMBON    35|02  878daa42bffffff  878daa42bffffff     TSEL   

     country  latitude  longitude               nmprov  \
0  INDONESIA  -9.57647  119.44008  NUSA TENGGARA TIMUR   
1  INDONESIA   0.87258  103.46535       KEPULAUAN RIAU   
2  INDONESIA  -7.55466  110.50652          JAWA TENGAH   
3  INDONESIA   0.86748  103.46469       KEPULAUAN RIAU   
4  INDONESIA  -3.99663  104.81675     SUMATERA SELATAN   
5  INDONESIA  -6.17857  106.98569           JAWA BARAT   
6  INDONESIA  -7.43031  112.72959           JAWA TIMUR   
7  INDONESIA   1.19286   97.67726       SUMATERA UTARA   
8  INDONESIA  -6.16190  106.65754               BANTEN   
9  INDONESIA  -7.88850  111.37660           JAWA TIMUR   

                     nmkab                nmkec activity_prov activity_kab  \
0              SUMBA BARAT                 LOLI            53        53|01   
1                  KARIMUN                 BURU            21        21|01   
2                 BOYOLALI            TAMANSARI            33        33|09   
3                  KARIMUN                 BURU            21        21|01   
4  OGAN KOMERING ULU TIMUR          BELITANG II            16        16|09   
5                   BEKASI         MEDAN SATRIA            32        32|75   
6                 SIDOARJO              BUDURAN            35        35|15   
7             GUNUNGSITOLI  GUNUNGSITOLI IDANOI            12        12|78   
8                TANGERANG            BATUCEPER            36        36|71   
9                 PONOROGO               JAMBON            35        35|02   

  activity_kec            h3_10             h3_9             h3_7  \
0    53|01|050  8a95ad751a27fff  8995ad751a3ffff  8795ad751ffffff   
1    21|01|031  8a652083011ffff  89652083013ffff  876520830ffffff   
2    33|09|041  8a8d8c024727fff  898d8c02473ffff  878d8c024ffffff   
3    21|01|031  8a65208308c7fff  8965208308fffff  876520830ffffff   
4    16|09|080  8a8ca941d8dffff  898ca941d8fffff  878ca941dffffff   
5    32|75|061  8a8c104ce0b7fff  898c104ce0bffff  878c104ceffffff   
6    35|15|120  8a8d842d8557fff  898d842d857ffff  878d842d8ffffff   
7    12|78|010  8a657475cc9ffff  89657475ccbffff  87657475cffffff   
8    36|71|050  8a8c106f3997fff  898c106f39bffff  878c106f3ffffff   
9    35|02|130  8a8daa40cad7fff  898daa40cafffff  878daa40cffffff   

          h3_7_poi poi_id   poi       start_datetime         end_datetime  \
0  8795ad751ffffff   None  Non

In [ ]:
from pyspark.sql import functions as F

def run_simple_sanity_check(df, execute_month=None):
    """
    Run a simple sanity check counting rows by suffix and event_month
    Parameters:
        df: DataFrame to check
        execute_month: Optional filter for specific execute_month (format: YYYYMM)
    """
    # Filter by execute_month if provided
    if execute_month:
        df = df.filter(F.col('execute_month') == execute_month)
        print(f"\nFiltering for execute_month: {execute_month}")

    # Get total count
    total_rows = df.count()
    print(f"\nTotal rows: {total_rows:,}")
    
    # Count by suffix
    print("\nCount by suffix:")
    df.groupBy('suffix') \
      .count() \
      .orderBy('suffix') \
      .withColumn('percentage', F.round(F.col('count') * 100 / total_rows, 2)) \
      .show()

    # Count by event_month
    print("\nCount by event_month:")
    df.groupBy('event_month') \
      .count() \
      .orderBy('event_month') \
      .withColumn('percentage', F.round(F.col('count') * 100 / total_rows, 2)) \
      .show()


def main():
    # Read the table
    df = spark.read.table('pnt_bps_int_stg.data_cerdas_mobility_trip_sample')
    
    # Get current execute_month (you might want to modify this logic)
    current_execute_month = '2024-12'
    
    # Run sanity checks for specific execute_month
    results = run_simple_sanity_check(df, execute_month=current_execute_month)
    
    # Optional: Save results to a file
    # with open(f'sanity_check_results_{current_execute_month}.json', 'w') as f:
    #     json.dump(results, f, indent=2)

if __name__ == "__main__":
    main()